In [5]:
!pip install mido

In [6]:
import mido
from mido import MidiFile, MidiTrack, Message
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
import numpy as np

##Load MID files

In [7]:
!unzip /content/chillhopdata.zip -d chillhop

Archive:  /content/chillhopdata.zip
  inflating: chillhop/1.mid          
  inflating: chillhop/10.mid         
  inflating: chillhop/11.mid         
  inflating: chillhop/12.mid         
  inflating: chillhop/13.mid         
  inflating: chillhop/14.mid         
  inflating: chillhop/15.mid         
  inflating: chillhop/16.mid         
  inflating: chillhop/17.mid         
  inflating: chillhop/18.mid         
  inflating: chillhop/19.mid         
  inflating: chillhop/2.mid          
  inflating: chillhop/20.mid         
  inflating: chillhop/3.mid          
  inflating: chillhop/4.mid          
  inflating: chillhop/5.mid          
  inflating: chillhop/6.mid          
  inflating: chillhop/7.mid          
  inflating: chillhop/8.mid          
  inflating: chillhop/9.mid          
  inflating: chillhop/Cymatics - Eternity MIDI 1 - C Maj.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 10 - F Min.mid  
  inflating: chillhop/Cymatics - Eternity MIDI 11 - A Maj.mid  
  inflating: 

In [8]:
import os

#store all the notes
notes = []

for song in os.listdir("/content/chillhop"):
  mid = MidiFile("/content/chillhop/" + song)
  
  for msg in mid:
    if not msg.is_meta and msg.channel == 0 and msg.type == "note_on":
      data = msg.bytes()
      notes.append(data[1]) 

##Scale Data

In [9]:
scaler = MinMaxScaler()
notes = list(scaler.fit_transform(np.array(notes).reshape(-1,1)))

##Create Train Data

In [10]:
notes = [list(note) for note in notes]

X = []
y = []

n_prev = 30

for i in range(len(notes) - n_prev):
  X.append(notes[i:i+n_prev])
  y.append(notes[i+n_prev])

X_test = X[-300:]
X = X[:-300]
y= y[:-300]

##Build LSTM

In [19]:
model = Sequential([
  LSTM(256, input_shape = (n_prev,1), return_sequences= True),
  Dropout(0.6),
  LSTM(128, input_shape = (n_prev,1), return_sequences= True),
  Dropout(0.6),
  LSTM(64, input_shape = (n_prev,1), return_sequences= True),
  Dropout(0.6),
  LSTM(32, input_shape = (n_prev,1), return_sequences= False),
  Dropout(0.6),
  
  Dense(1, activation= "linear")
])

model.summary()
optimizer = Adam(lr = 0.001)
model.compile(loss = "mse" , optimizer = optimizer)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 256)           264192    
                                                                 
 dropout_3 (Dropout)         (None, 30, 256)           0         
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           197120    
                                                                 
 dropout_4 (Dropout)         (None, 30, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 30, 64)            49408     
                                                                 
 dropout_5 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_6 (LSTM)               (None, 32)               

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


##Train

In [20]:
model.fit(np.array(X), np.array(y), batch_size = 16, epochs = 15, verbose = 1)

Epoch 1/15
195/195 [==============================] - 7s 11ms/step - loss: 0.0276
Epoch 2/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0199
Epoch 3/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0186
Epoch 4/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0172
Epoch 5/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0166
Epoch 6/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0158
Epoch 7/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0153
Epoch 8/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0151
Epoch 9/15
195/195 [==============================] - 2s 13ms/step - loss: 0.0152
Epoch 10/15
195/195 [==============================] - 2s 12ms/step - loss: 0.0150
Epoch 11/15
195/195 [==============================] - 2s 11ms/step - loss: 0.0148
Epoch 12/15
195/195 [==============================] - 2s 12ms/step - loss: 0.0149
Epoch 13/15
1

##Generating and Saving Music

In [21]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

mid = MidiFile()
track = MidiTrack()
t = 0

for note in prediction:
  #147 means note_on
  #67 in velosity
  note = np.asarray([147, note, 67])
  bytes = note.astype(int)
  msg = Message.from_bytes(bytes[0:3])
  t += 1
  msg.time = t
  track.append(msg)

mid.tracks.append(track)
mid.save("LSTM_music.mid")
